In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from itertools import combinations_with_replacement

In [2]:
file_path = '../../../data/wage2015_subsample_inference.csv'
data = pd.read_csv(file_path)

print("Primeras filas del dataset:")
print(data.head())

Primeras filas del dataset:
   rownames       wage     lwage  sex  shs  hsg  scl  clg   ad   mw  ...   we  \
0        10   9.615385  2.263364  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   
1        12  48.076923  3.872802  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   
2        15  11.057692  2.403126  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0   
3        18  13.942308  2.634928  1.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0   
4        19  28.846154  3.361977  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   

    ne  exp1  exp2    exp3     exp4     occ  occ2     ind  ind2  
0  1.0   7.0  0.49   0.343   0.2401  3600.0    11  8370.0    18  
1  1.0  31.0  9.61  29.791  92.3521  3050.0    10  5070.0     9  
2  1.0  18.0  3.24   5.832  10.4976  6260.0    19   770.0     4  
3  1.0  25.0  6.25  15.625  39.0625   420.0     1  6990.0    12  
4  1.0  22.0  4.84  10.648  23.4256  2015.0     6  9470.0    22  

[5 rows x 21 columns]


In [3]:
# Verificar si hay valores faltantes en cada columna
missing_values = data.isnull().sum()

print("\nNúmero de valores faltantes por columna:")
print(missing_values)


Número de valores faltantes por columna:
rownames    0
wage        0
lwage       0
sex         0
shs         0
hsg         0
scl         0
clg         0
ad          0
mw          0
so          0
we          0
ne          0
exp1        0
exp2        0
exp3        0
exp4        0
occ         0
occ2        0
ind         0
ind2        0
dtype: int64


In [4]:
# Calcular las estadísticas descriptivas
descriptive_stats = data.describe(percentiles=[0.25, 0.5, 0.75])

print("Estadísticas descriptivas del dataset:")
print(descriptive_stats)

Estadísticas descriptivas del dataset:
           rownames         wage        lwage          sex          shs  \
count   5150.000000  5150.000000  5150.000000  5150.000000  5150.000000   
mean   15636.346602    23.410410     2.970787     0.444466     0.023301   
std     9700.744395    21.003016     0.570385     0.496955     0.150872   
min       10.000000     3.021978     1.105912     0.000000     0.000000   
25%     7132.500000    13.461538     2.599837     0.000000     0.000000   
50%    15260.000000    19.230769     2.956512     0.000000     0.000000   
75%    24337.750000    27.777778     3.324236     1.000000     0.000000   
max    32643.000000   528.845673     6.270697     1.000000     1.000000   

               hsg          scl          clg           ad           mw  ...  \
count  5150.000000  5150.000000  5150.000000  5150.000000  5150.000000  ...   
mean      0.243883     0.278058     0.317670     0.137087     0.259612  ...   
std       0.429465     0.448086     0.465616    

In [5]:
conditions = [
    data['shs'] == 1,
    data['hsg'] == 1,
    data['scl'] == 1,
    data['clg'] == 1,
    data['ad'] == 1
]
choices = ['shs', 'hsg', 'scl', 'clg', 'ad']

data['education'] = np.select(conditions, choices, default=np.nan)

# Calcular el percentil 75 del logaritmo del salario (lwage) 
lwage_75th_percentile = data['lwage'].quantile(0.75)

# Filtrar los datos para obtener a las mujeres con grado universitario o superior, y lwage en el 25% más alto
filtered_data = data[
    (data['sex'] == 1) & 
    (data['education'].isin(['clg', 'ad'])) & 
    (data['lwage'] >= lwage_75th_percentile)
]

# Contar el número de mujeres que cumplen las condiciones
num_women = len(filtered_data)

# Mostrar el dataframe correspondiente y el resultado
print(f"Número de mujeres con grado universitario o superior en el 25% más rico (basado en lwage): {num_women}")

Número de mujeres con grado universitario o superior en el 25% más rico (basado en lwage): 419


In [6]:
conditions = [
    data['shs'] == 1,
    data['hsg'] == 1,
    data['scl'] == 1,
    data['clg'] == 1,
    data['ad'] == 1
]
choices = ['shs', 'hsg', 'scl', 'clg', 'ad']

data['education'] = np.select(conditions, choices, default=np.nan)

# Calcular el percentil 75 del logaritmo del salario (lwage)
lwage_75th_percentile = data['lwage'].quantile(0.75)

# Filtrar los datos para obtener a los hombres con grado de high school (hsg) o menor, y lwage en el 25% más alto
filtered_data = data[
    (data['sex'] == 0) &  # 0 corresponde a hombres
    (data['education'].isin(['shs', 'hsg'])) & 
    (data['lwage'] >= lwage_75th_percentile)
]

# Contar el número de hombres que cumplen las condiciones
num_men = len(filtered_data)

# Mostrar el resultado
print(f"Número de hombres con grado de high school o menor en el 25% más rico (basado en lwage): {num_men}")


Número de hombres con grado de high school o menor en el 25% más rico (basado en lwage): 118


In [7]:
data = pd.read_csv(file_path)

# Crear el DataFrame con solo la variable log(wage)
df_log_wage = data[['lwage']]

# Crear el DataFrame con todas las variables excepto las relacionadas con el salario
# Primero identificamos las columnas relacionadas con el salario
wage_related_columns = ['wage', 'lwage']

# Excluimos esas columnas del DataFrame original
df_non_wage_related = data.drop(columns=wage_related_columns)

# Mostrar los DataFrames resultantes
print("DataFrame con solo log(wage):")
print(df_log_wage.head())

print("\nDataFrame sin las variables relacionadas con el salario:")
print(df_non_wage_related.head())



DataFrame con solo log(wage):
      lwage
0  2.263364
1  3.872802
2  2.403126
3  2.634928
4  3.361977

DataFrame sin las variables relacionadas con el salario:
   rownames  sex  shs  hsg  scl  clg   ad   mw   so   we   ne  exp1  exp2  \
0        10  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0   7.0  0.49   
1        12  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  31.0  9.61   
2        15  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  18.0  3.24   
3        18  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  25.0  6.25   
4        19  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  22.0  4.84   

     exp3     exp4     occ  occ2     ind  ind2  
0   0.343   0.2401  3600.0    11  8370.0    18  
1  29.791  92.3521  3050.0    10  5070.0     9  
2   5.832  10.4976  6260.0    19   770.0     4  
3  15.625  39.0625   420.0     1  6990.0    12  
4  10.648  23.4256  2015.0     6  9470.0    22  


In [8]:
Y_array = data['lwage'].to_numpy()
# Mostrar el array resultante
print("Array para la variable Y (log(wage)):")
print(Y_array)

Array para la variable Y (log(wage)):
[2.26336438 3.87280229 2.40312632 ... 3.64965874 3.49550806 2.85115104]


### Regresiones

In [9]:
## Modelo Básico

# Cargar el dataset
data = pd.read_csv(file_path)

df_lwage = data[['lwage']]

# Variables básicas: sexo, polinomios de experiencia y otras
basic_vars = ['sex', 'exp1','hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne']

data_with_dummies = pd.get_dummies(data, columns=['occ2', 'ind2'], drop_first=True)

dummy_vars = [col for col in data_with_dummies.columns if col.startswith('occ2_') or col.startswith('ind2_')]
basic_vars_with_dummies = basic_vars + dummy_vars

# Paso 3: Crear el array para Y (variable dependiente lwage)
Y = df_lwage.values.flatten()

# Paso 4: Crear el array para los predictores del modelo básico
X_basic_model = data_with_dummies[basic_vars_with_dummies].values

# Paso 5: Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, Y_train, Y_test = train_test_split(X_basic_model, Y, test_size=0.2, random_state=42)

# Paso 6: Estimar el modelo de regresión lineal en el conjunto de entrenamiento
model = LinearRegression()
model.fit(X_train, Y_train)

# Paso 7: Predecir sobre el conjunto de prueba
Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)

# Paso 8: Evaluar el modelo
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

mse_train = mean_squared_error(Y_train, Y_pred_train)
r2_train = r2_score(Y_train, Y_pred_train)

n = len(Y_test)
k = X_test.shape[1] 

r2_adj_test = 1 - (1 - r2) * (n - 1) / (n - k - 1)
r2_adj_train = 1 - (1 - r2_train) * (len(Y_train) - 1) / (len(Y_train) - k - 1)

# Resultados
print(f"Error Cuadrático Medio (MSE) Modelo Test: {mse}")
print(f"Coeficiente de determinación (R²) Modelo Test : {r2}")
print(f"Coeficiente de determinación ajustado (R² ajustado) Modelo Test: {r2_adj_test}")

print(f"Error Cuadrático Medio (MSE) Modelo Train: {mse_train}")
print(f"Coeficiente de determinación (R²) Modelo Train: {r2_train}")
print(f"Coeficiente de determinación ajustado (R² ajustado) Modelo Train: {r2_adj_train}")

Error Cuadrático Medio (MSE) Modelo Test: 0.2363633528736828
Coeficiente de determinación (R²) Modelo Test : 0.278993506970851
Coeficiente de determinación ajustado (R² ajustado) Modelo Test: 0.24216988628499048
Error Cuadrático Medio (MSE) Modelo Train: 0.22224656834936976
Coeficiente de determinación (R²) Modelo Train: 0.3154005637177951
Coeficiente de determinación ajustado (R² ajustado) Modelo Train: 0.30698818430906816


In [10]:
## Modelo Flexible

# Cargar el dataset
data = pd.read_csv(file_path)

df_lwage = data[['lwage']]

# Variables básicas: sexo, polinomios de experiencia y otras
basic_vars = ['sex', 'exp1','exp2','exp3','exp4','hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne']

exp_vars = ['exp1', 'exp2', 'exp3', 'exp4']

interaction_vars = ['hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne']

data_with_dummies = pd.get_dummies(data, columns=['occ2', 'ind2'], drop_first=True)

dummy_vars = [col for col in data_with_dummies.columns if col.startswith('occ2_') or col.startswith('ind2_')]
flexible_vars_with_dummies = basic_vars + dummy_vars

for exp_var in exp_vars:
    for var in interaction_vars:
        interaction_term = f"{exp_var}_x_{var}"
        data_with_dummies[interaction_term] = data_with_dummies[exp_var] * data_with_dummies[var]

interaction_terms = [f"{exp_var}_x_{var}" for exp_var in exp_vars for var in interaction_vars]
flexible_vars_with_dummies += interaction_terms

# Paso 3: Crear el array para Y (variable dependiente lwage)
Y = df_lwage.values.flatten()

# Paso 4: Crear el array para los predictores del modelo básico
X_flexible_model = data_with_dummies[flexible_vars_with_dummies].values

# Paso 5: Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, Y_train, Y_test = train_test_split(X_flexible_model, Y, test_size=0.2, random_state=42)

# Paso 6: Estimar el modelo de regresión lineal en el conjunto de entrenamiento
model = LinearRegression()
model.fit(X_train, Y_train)

# Paso 7: Predecir sobre el conjunto de prueba
Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)

# Paso 8: Evaluar el modelo
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

mse_train = mean_squared_error(Y_train, Y_pred_train)
r2_train = r2_score(Y_train, Y_pred_train)

n = len(Y_test)
k = X_test.shape[1] 

r2_adj_test = 1 - (1 - r2) * (n - 1) / (n - k - 1)
r2_adj_train = 1 - (1 - r2_train) * (len(Y_train) - 1) / (len(Y_train) - k - 1)

# Resultados
print(f"Error Cuadrático Medio (MSE) Modelo Test: {mse}")
print(f"Coeficiente de determinación (R²) Modelo Test : {r2}")
print(f"Coeficiente de determinación ajustado (R² ajustado) Modelo Test: {r2_adj_test}")

print(f"Error Cuadrático Medio (MSE) Modelo Train: {mse_train}")
print(f"Coeficiente de determinación (R²) Modelo Train: {r2_train}")
print(f"Coeficiente de determinación ajustado (R² ajustado) Modelo Train: {r2_adj_train}")

Error Cuadrático Medio (MSE) Modelo Test: 0.23302171432883392
Coeficiente de determinación (R²) Modelo Test : 0.28918689380049256
Coeficiente de determinación ajustado (R² ajustado) Modelo Test: 0.2284528625745853
Error Cuadrático Medio (MSE) Modelo Train: 0.21923222710629245
Coeficiente de determinación (R²) Modelo Train: 0.3246858198686525
Coeficiente de determinación ajustado (R² ajustado) Modelo Train: 0.311139398721887


In [11]:
## Modelo Extra Flexible

# Load data
data = pd.read_csv(file_path)

df_lwage = data[['lwage']]

# Variables de experiencia y de interacción
exp_vars = ['exp1', 'exp2', 'exp3', 'exp4']
interaction_vars = ['hsg', 'scl', 'clg', 'ad', 'so', 'we', 'ne']

# Crear dummies para occ2 e ind2
data_with_dummies = pd.get_dummies(data, columns=['occ2', 'ind2'], drop_first=True)

# Añadir las dummies a las variables de interacción
dummy_vars = [col for col in data_with_dummies.columns if col.startswith('occ2_') or col.startswith('ind2_')]
interaction_vars += exp_vars + dummy_vars

# Crear interacciones cuadráticas (combinaciones de 2 variables con reemplazo)
interaction_terms = []
for var1, var2 in combinations_with_replacement(interaction_vars, 2):
    interaction_term = f"{var1}_x_{var2}"
    data_with_dummies[interaction_term] = data_with_dummies[var1] * data_with_dummies[var2]
    interaction_terms.append(interaction_term)

# Crear el array para los predictores del modelo
basic_vars_with_dummies = interaction_vars + interaction_terms
X_extra_flexible_model = data_with_dummies[basic_vars_with_dummies].values

# Preparar la variable de respuesta
Y = df_lwage.values.flatten()

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X_extra_flexible_model, Y, test_size=0.2, random_state=42)

# Estimar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, Y_train)

# Predecir 
Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)

# Evaluar el modelo
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

mse_train = mean_squared_error(Y_train, Y_pred_train)
r2_train = r2_score(Y_train, Y_pred_train)

n = len(Y_test)
k = X_test.shape[1] 

r2_adj_test = 1 - (1 - r2) * (n - 1) / (n - k - 1)
r2_adj_train = 1 - (1 - r2_train) * (len(Y_train) - 1) / (len(Y_train) - k - 1)



C:\Users\Usuario\AppData\Local\Temp\ipykernel_16948\1062407842.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_with_dummies[interaction_term] = data_with_dummies[var1] * data_with_dummies[var2]
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16948\1062407842.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_with_dummies[interaction_term] = data_with_dummies[var1] * data_with_dummies[var2]
C:\Users\Usuario\AppData\Local\Temp\ipykernel_16948\1062407842.py:24: PerformanceWarning: DataFrame is highly fragmented.  Thi

Error Cuadrático Medio (MSE) Modelo Test: 0.32061917525178557
Coeficiente de determinación (R²) Modelo Test : 0.02197821982272763
Coeficiente de determinación ajustado (R² ajustado) Modelo Test: 3.509686812474846
Error Cuadrático Medio (MSE) Modelo Train: 0.17194607086402067
Coeficiente de determinación (R²) Modelo Train: 0.47034420347313144
Coeficiente de determinación ajustado (R² ajustado) Modelo Train: 0.18867525998729195


In [13]:
# Resultados
print(f"Error Cuadrático Medio (MSE) Modelo Test: {mse}")
print(f"Coeficiente de determinación (R²) Modelo Test : {r2}")
print(f"Coeficiente de determinación ajustado (R² ajustado) Modelo Test: {r2_adj_test}")

print(f"Error Cuadrático Medio (MSE) Modelo Train: {mse_train}")
print(f"Coeficiente de determinación (R²) Modelo Train: {r2_train}")
print(f"Coeficiente de determinación ajustado (R² ajustado) Modelo Train: {r2_adj_train}")

Error Cuadrático Medio (MSE) Modelo Test: 0.32061917525178557
Coeficiente de determinación (R²) Modelo Test : 0.02197821982272763
Coeficiente de determinación ajustado (R² ajustado) Modelo Test: 3.509686812474846
Error Cuadrático Medio (MSE) Modelo Train: 0.17194607086402067
Coeficiente de determinación (R²) Modelo Train: 0.47034420347313144
Coeficiente de determinación ajustado (R² ajustado) Modelo Train: 0.18867525998729195
